## Convolutional Neural Network
- MNIST data
- 3 Convolutional layers
- 2 fully conected layers

## 1. Settings

### 1) Import required libraries

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

### 2) Set hyperparameters

In [10]:
batch_size = 16
learning_rate = 0.0002
num_epoch = 10

## 2. Data

### 1) Download Data

In [4]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

### 2) Check Dataset

In [5]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
mnist_test.__getitem__(0)[0].size(), mnist_test.__len__()

torch.Size([1, 28, 28]) 60000


(torch.Size([1, 28, 28]), 10000)

### 3) Set DataLoader

In [6]:
train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size, shuffle=True,num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size, shuffle=False,num_workers=2,drop_last=True)

## 3. Model & Optimizer

### 1) CNN Model

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        
        return out
    
model = CNN()
        

In [12]:
### 2) Loss func & Optimizer
loss_func = nn.CrossEntropyLoss()
optimizer =  torch.optim.SGD(model.parameters(), lr = learning_rate)

## 4. Train

In [13]:
for i in range(num_epoch):
    for j,[image, label] in enumerate(train_loader):
        x = Variable(image)
        y_ = Variable(label)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
        if j % 1000 ==0:
            print(loss)

tensor(2.3018, grad_fn=<NllLossBackward>)
tensor(2.3056, grad_fn=<NllLossBackward>)
tensor(2.2933, grad_fn=<NllLossBackward>)
tensor(2.3036, grad_fn=<NllLossBackward>)
tensor(2.2970, grad_fn=<NllLossBackward>)
tensor(2.2929, grad_fn=<NllLossBackward>)
tensor(2.3001, grad_fn=<NllLossBackward>)
tensor(2.3013, grad_fn=<NllLossBackward>)
tensor(2.2822, grad_fn=<NllLossBackward>)
tensor(2.2917, grad_fn=<NllLossBackward>)
tensor(2.2774, grad_fn=<NllLossBackward>)
tensor(2.3021, grad_fn=<NllLossBackward>)
tensor(2.2922, grad_fn=<NllLossBackward>)
tensor(2.2907, grad_fn=<NllLossBackward>)
tensor(2.3234, grad_fn=<NllLossBackward>)
tensor(2.2763, grad_fn=<NllLossBackward>)
tensor(2.2943, grad_fn=<NllLossBackward>)
tensor(2.2977, grad_fn=<NllLossBackward>)
tensor(2.2692, grad_fn=<NllLossBackward>)
tensor(2.2824, grad_fn=<NllLossBackward>)
tensor(2.2653, grad_fn=<NllLossBackward>)
tensor(2.2978, grad_fn=<NllLossBackward>)
tensor(2.2750, grad_fn=<NllLossBackward>)
tensor(2.2718, grad_fn=<NllLossBac

## 5. Test

In [14]:
correct = 0
total = 0

for image, label in test_loader:
    x = Variable(image, volatile=True)
    y_ = Variable(label)
    
    output = model.forward(x)
    _, output_index = torch.max(output,1)
    
    total += label.size(0)
    correct += (output_index == y_).sum().float()
    
print("Accuracy of Test Data: {}".format(100*correct/total))

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """


Accuracy of Test Data: 87.72000122070312
